In [15]:
import os

In [16]:
%pwd

'/Users/omariko/Documents/GitHub'

In [7]:
os.chdir("../")

In [8]:
%pwd

'/Users/omariko/Documents/GitHub'

In [9]:
from dataclasses import dataclass
from pathlib import Path

@dataclass(frozen = True)
class DataTransformationConfig:
    root_dir: Path
    data_path: Path
    tokenizer_name: Path

In [10]:
from TextSummarizer.constants import CONFIG_FILE_PATH, PARAMS_FILE_PATH
from TextSummarizer.utils.common import read_yaml, create_directories

In [11]:
class ConfigurationManager:
    def __init__(self,
                 config_filepath = CONFIG_FILE_PATH,
                 params_filepath = PARAMS_FILE_PATH):
        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)

    def get_data_transformation_config(self) -> DataTransformationConfig:
        config = self.config.data_transformation

        create_directories([self.config.artifacts_root])

        data_transformation_config = DataTransformationConfig(
            root_dir = config.root_dir,
            data_path = config.data_path,
            tokenizer_name = config.tokenizer_name
        )

        return data_transformation_config

In [12]:
from TextSummarizer.logging import logger
from transformers import AutoTokenizer
from datasets import load_from_disk

/Users/omariko/Documents/GitHub/NLP-Text-Summarizer/venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[2025-02-27 18:55:12,867: INFO: config: PyTorch version 2.6.0 available.]


In [13]:
class DataTansformation:
    def __init__(self, config = DataTransformationConfig):
        self.config = config
        self.tokenizer = AutoTokenizer.from_pretrained(config.tokenizer_name)

    def convert_examples_to_features(self, example_batch):
        input_encodings = self.tokenizer(example_batch['dialogue'], max_length = 1024, truncation = True)

        with self.tokenizer.as_target_tokenizer():
            target_encodings = self.tokenizer(example_batch['summary'], max_length = 128, truncation = True)

        return {
            "input_ids": input_encodings['input_ids'],
            "attention_mask": input_encodings['attention_mask'],
            "labels": target_encodings['input_ids']
        }
    
    def convert(self):
        dataset_samsum = load_from_disk(self.config.data_path)
        dataset_samsum_pt = dataset_samsum.map(self.convert_examples_to_features, batched = True)
        dataset_samsum_pt.save_to_disk(os.path.join(self.config.root_dir, "samsum_dataset"))

In [14]:
try:
    config = ConfigurationManager()
    data_transformation_config = config.get_data_transformation_config()
    data_transformation = DataTansformation(config = data_transformation_config)
    data_transformation.convert()
except Exception as e:
    raise e

FileNotFoundError: [Errno 2] No such file or directory: 'config/config.yaml'